<a href="https://colab.research.google.com/github/dhu16/fortune_sentiments/blob/main/Week8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install PyDrive

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [5]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
%matplotlib inline
import pandas as pd
import string
import nltk
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14)

In [7]:
downloaded = drive.CreateFile({'id':"1O6RaEYB3Pu8y1IyLbJPZTSRTN9ekgN7-"})
downloaded.GetContentFile('fortune_phrases.xlsx')

In [13]:
#Create dataframe with only 1 column containing the sentences
df = pd.read_excel('fortune_phrases.xlsx', usecols=['phrase'])

#Convert dataframe to list
content=df.stack().tolist()
content[0:10]

['reflect on the consequences of your actions, impulsive choices are merely distractions',
 'find your passion and figure out how to fullfil it. All you really need is some focus and your wit ',
 "directness, spontaneity and independence are admirable qualities which you posses, be kind to those who don't for that is the test",
 'remember that patience is a virtue, in due time you will get what you pursue',
 'put your fears behind you and walk your own path. Look ahead of you and wait for a flash',
 'approach your tasks with wisdom and assurance, everything in life is a test of your endurance',
 'a sincere and loving person will present themselves to you as a loyal friend. There is no friend better than one who you vet',
 'you know or you will be approached by a person who is kind and wise, keep your mind open for what they may advise',
 'there will soon be a change to your material wealth and possessions, or your body and health, be catious this change may be stealth',
 'change your l

In [14]:
#Using nltk for stop words
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def full_remove(x, removal_list):
    for w in removal_list:
        x = x.replace(w, ' ')
    return x

#Remove punctuation
removePunc = [full_remove(x, list(string.punctuation)) for x in content]

#Make everything lower-case and remove any white space
sents_lower = [x.lower() for x in removePunc]
sents_lower = [x.strip() for x in sents_lower]

#Remove stop words
stops = stopwords.words("english")

def removeStopWords(stopWords, txt):
    newtxt = ' '.join([word for word in txt.split() if word not in stopWords])
    return newtxt

stop_set = ['the', 'a', 'an', 'i', 'he', 'she', 'they', 'to', 'of', 'it', 'from']
    
processedSentences = [removeStopWords(stop_set,x) for x in sents_lower]
processedSentences[0:20]

In [17]:
#Using TextBlob to determine polarity and subjectivity of sentences
from textblob import TextBlob

#Create polarity function and subjectivity function
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

#Finds all words that can be assinged polarity and subjectivity and averages them together for the whole sentence's respective polarity/subjectivity
pol_list = [pol(x) for x in processedSentences]
sub_list = [sub(x) for x in processedSentences]

In [19]:
#Append polarity and subjectivity sentiments to dataframe
df['Polarity']=pol_list
df['Subjectivity']=sub_list

#Print dataframe
pd.set_option('display.max_colwidth', -1)
df.sample(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,phrase,Polarity,Subjectivity
43,pay attention for someone in your vicinity; will present you with a new idea or opportunity,0.068182,0.227273
97,"rely on your survival instincts, your good sense and your ability to react in this next phase, you have been preparing for this in many ways",0.400000,0.366667
88,"a start of a deep connection or new relationship, may be romantic or platonic, awaits you, a meeting of the spiritis has been long overdue",0.021591,0.438636
103,"soon your courage and tenacity will be tested, this will determine your long-term success",0.125000,0.200000
79,"a new source of inspiration will enter yout life, stay open to new ideas or a new enterprise",0.102273,0.465909
96,"there is a ending, a conclusion or a loss in your near future, that you can be sure of",0.200000,0.471296
42,"a woman in your life will approach you she represents, an end to sorrows, a warning or a test",0.000000,0.000000
29,"your weakness is that you are unable to accept loss, resilience is adapting to whatever the fates may toss",-0.500000,0.500000
12,"you seek the contentment of the heart, you may be getting closer or you may be worlds apart",0.000000,0.000000
80,"you may be feeling hopeless and trapped by anguish, you should seek an emotional or creative outlet",0.100000,0.550000
